# Ocean Plastic Modelling Tutorial

# Overview

This tutorial focuses on modelling the motion of plastic in the oceans. Such models are created by non-profits like The Ocean Cleanup to help them make informed decisions about where to focus their cleanup efforts and what methods could be most effective.

Fluid dynamics will be an important topic in this tutorial, with concepts like advection, Eulerian and Lagrangian methods and (of course) numerical integration being central in these models.

We will recreate the 'advection scheme' behind the Python library 'Ocean Parcels', then learn to use the library in some simple situations and see some more realistic use cases as laid out by Ocean Parcels' documentation.

# Advection
This notion is key in ocean plastic modelling because it describes exactly the behaviour we want to model. Advection is the transport of something by the motion of a fluid into which it is introduced. In the case of ocean plastic, the plastics whose motion we are trying to predict are advected by ocean currents and wind.

Now, to put concrete maths to the idea; the 'advection operator' is $ \textbf{u} \cdot \nabla = u_x \frac{\partial}{\partial x} + u_y \frac{\partial}{\partial y} + u_z \frac{\partial}{\partial z} = \frac{d}{dt}-\frac{\partial}{\partial t}$.
Clearly it describes a rate of change with respect to time and due to the presence of the velocity vector in the operator, it's apparent that the operator picks out the rate of change caused by the motion of the fluid. The '$\textbf{u}$' here is the velocity field of the fluid - $\textbf{u}(\textbf{x},t)$ is a more exact way of writing it because it is a vector function of position and time, but it is commonly shortened to $\textbf{u}$.

If we have a plastic particle in the fluid at position $\textbf{r}=x \mathbf{\hat{x}} + y \mathbf{\hat{y}} + z \mathbf{\hat{z}}$, advection causes it to have velocity $(\textbf{u} \cdot \nabla) \space \textbf{r} = \textbf{u}(\textbf{r},t)$. Thus, if we know $\textbf{r}(t_0)$ we can calculate the position of the particle at any later stage by

$\textbf{r}(t') = \int \limits_{t_0}^t' dt \space \textbf{u}(\textbf{r}(t),t) + \textbf{r}(t_0)$

We will work through numerically integrating this to get trajectories in this tutorial, but first we must discuss how the velocity field and particle positions are represented in the model.

# Euler vs. Lagrange

There are two main ways of considering fields in compuational fluid dynamics - the Eulerian and Lagrangian viewpoints.

The Eulerian viewpoint considers a grid dividing up the space into cells. Each cell then has a set of values associated with it, representing the values of the fluid's properties. Numerically integrating the system then involves repeatedly updating the values of the fluid's properties in each cell. Taking the grid to be made up of infinitely small cells gives an exact description of the fluid, but on a computer we have to decide on cells that are 'small enough'.

The Lagrangian viewpoint of a fluid considers it as a collection of many parcels of the fluid. Numerically integrating the system then involves repeatedly updating the positions of the parcels and the values of the fluids properties in a given parcel. In this case, an exact description of the fluid would involve taking infinitely maky parcels but we have to content ourselves with a 'big enough' number of parcels in practice.

In theory, either viewpoint effectively describes the fluid - but we have to decide on our approach before we start working on our problem. The approach that will be taken here is as follows:

- The velocity field $\textbf{u}(\textbf{x},t)$ will be represented by an Eulerian field.

- The fluid of plastic particles will be represented by a Lagrangian set of parcels. $\textbf{r}(\textbf{x}_0,t)$ is the position (at time $t$) of the parcel that was at $\textbf{x}_0$ at time $t_0$.

Although this is the way that the problem was set up in the previous section '**Advection**', note that we could equivalently have set up the problem using only the Eulerian or only the Lagrangian specification of the fluids. If only the Eulerian specification was used, an additional property of the fluid, 'plastic density' would be associated with each cell in the Eulerian picture and we could evolve that to replicate the dynamics of plastics in the ocean.

Alternatively, if only the Lagrangian specification was used, the velocity field $\textbf{u}(\textbf{x},t)$ that describes ocean currents would be replaced by $\frac{d}{dt}\textbf{X}(\textbf{x}_0,t)$ - the time derivative of trajectories of fluid parcels, where these fluid parcels are seperate to the plastic parcels and represent instead moving portions of the ocean.

We use the Eulerian specification for the velocity field and the Lagrangian specification for the plastic trajectories in this tutorial because that is the setup used in the Ocean Parcels library. The choice is advantageous because it allows for existing datasets of ocean currents in the Eulerian specification to be used to produce results for the trajectories of discrete particles which are most intuitively visualised in the Lagrangian specification.

# Coding the advection of fluid parcels in an Eulerian specified field

As mentioned earlier, this is the workhorse behind the models in the Ocean Parcels library. Before we start calling it from the library, let's figure out how we could create it ourselves.

Run the following cell - it imports the necessary libraries and sets up an Eulerian specification of a spiraling time-independent velocity field in a 2D square region broken into a 20$\times$20 grid:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the grid
grid_size = 20
x = np.linspace(0, 1, grid_size)
y = np.linspace(0, 1, grid_size)
X, Y = np.meshgrid(x, y)

# Define a spiraling velocity field (time-independent)
U = -(Y - 0.5)  # x-component of velocity
V = (X - 0.5)   # y-component of velocity

# Plot the velocity field
plt.figure(figsize=(4, 4))
plt.quiver(X, Y, U, V)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Velocity Field')
plt.show()

Also run this cell to set up the tutorial's continuous feedback system:

In [ ]:
# @title
from IPython.core.magic import register_cell_magic

# Function to perform all checks
def perform_checks_1():
    # Check time_step
    if 'time_step' in globals():
        if time_step == 0.1:
            print("✅ time_step is initialised correctly")
        else:
            print("❌ time_step is not initialised correctly")
    else:
        print("❌ time_step is not defined")

    # Check num_steps
    if 'num_steps' in globals():
        if num_steps == 30:
            print("✅ num_steps is initialised correctly")
        else:
            print("❌ num_steps is not initialised correctly")
    else:
        print("❌ num_steps is not defined")

    # Check parcel_trajectory
    if 'parcel_trajectory' in globals():
        corr_obj = np.zeros((num_steps + 1, 2))
        corr_obj[0] = np.array([0.6, 0.4])
        if (parcel_trajectory == corr_obj).all():
            print("✅ parcel_trajectory is initialised correctly")
        else:
            print("❌ parcel_trajectory is not initialised correctly")
    else:
        print("❌ parcel_trajectory is not defined")

# Define the cell magic function
@register_cell_magic
def check_all_1(line, cell):
    # Execute the student's code
    exec(cell, globals())
    # Run checks
    perform_checks_1()

# Define the cell magic function for the final trajectory
@register_cell_magic
def check_all_2(line, cell):
    # Execute the student's code
    exec(cell, globals())

    # Calculate the expected trajectory
    expected_trajectory = np.zeros((num_steps + 1, 2))
    expected_trajectory[0] = np.array([0.6, 0.4])

    # Function to get velocity in the grid cell where the parcel currently is:
    def get_velocity(position, U, V, x, y):
        # Find the nearest indices in the grid
        ix = np.argmin(np.abs(x - position[0]))
        iy = np.argmin(np.abs(y - position[1]))
        return np.array([U[iy, ix], V[iy, ix]])

    # Runge-Kutta integration
    for n in range(30):
        r_n = expected_trajectory[n]

        k1 = get_velocity(r_n, U, V, x, y)
        k2 = get_velocity(r_n + 0.1 * k1 / 2, U, V, x, y)
        k3 = get_velocity(r_n + 0.1 * k2 / 2, U, V, x, y)
        k4 = get_velocity(r_n + 0.1 * k3, U, V, x, y)

        r_next = r_n + (0.1 / 6) * (k1 + 2 * k2 + 2 * k3 + k4)

        # Update the trajectory
        expected_trajectory[n + 1] = r_next

    # Perform checks
    if 'parcel_trajectory' in globals():
        if (parcel_trajectory == expected_trajectory).all():
            print("✅ parcel_trajectory is correct.")
        else:
            # Optionally, print out the discrepancy for debugging
            discrepancy = parcel_trajectory - expected_trajectory
            print("Discrepancy:", discrepancy)
            print("❌ parcel_trajectory is not correct. Discrepancy printed above.")
    else:
        print("❌ parcel_trajectory is not defined.")



Now, we want to set up a parcel whose trajectory we want to numerically integrate. We want to use a time step of 0.1 and we will run our numerical integration for 30 steps.

Define two variables: *time_step* (to store the size of the time step) and *num_steps* (to store the number of steps that will be taken), then define a numpy array of zeros, *parcel_trajectory*, that has shape (*num_steps* +1)$\times 2$.

To start our parcel at (0.6,0.4), let the zeroth row in the numpy array take that value.


In [ ]:
%%check_all_1 # Leave this here - it will give continuous feedback on your code
#----------------------------------------------------------------------------
#Start coding here:


Now we will carry out the numerical integration of the trajectory. We'll use the fourth-order Runge-Kutta method by creating a *for loop* that loops through the range of *num_steps*, with each iteration of the loop applying the following procedure:

$\textbf{r}(\textbf{x}_0)_{n+1} = \textbf{r}(\textbf{x}_0)_{n} + \frac{\Delta t}{6}(\textbf{k}_1 + 2\textbf{k}_2 +2\textbf{k}_3 + \textbf{k}_4)$, where $\textbf{k}_1 = \textbf{u}(\textbf{r}(\textbf{x}_0)_{n})$, $\textbf{k}_2 = \textbf{u}(\textbf{r}(\textbf{x}_0)_{n} + \Delta t \frac{\textbf{k}_1}{2})$, $\textbf{k}_3 = \textbf{u}(\textbf{r}(\textbf{x}_0)_{n} + \Delta t\frac{ \textbf{k}_2}{2})$, $\textbf{k}_4 = \textbf{u}(\textbf{r}(\textbf{x}_0)_{n} + \Delta t \space \textbf{k}_3)$.

To help you, a function that returns the value of the velocity field in any cell from the (x,y) position is given.

In [ ]:
%%check_all_2 # Leave this here - it will give continuous feedback on your code

# Function to get velocity in the grid cell where the parcel currently is:
def get_velocity(position, U, V, x, y):
    # Find the nearest indices in the grid
    ix = np.argmin(np.abs(x - position[0]))
    iy = np.argmin(np.abs(y - position[1]))
    return np.array([U[iy, ix], V[iy, ix]])
# Example usage: get_velocity([0.6,0.4], U, V, x, y) returns velocity at
# the parcel's start point.
#----------------------------------------------------------------------------
# Start coding here:


Run the following code to view the parcel's trajectory:

In [ ]:
# Plot the velocity field
plt.figure(figsize=(4, 4))
plt.quiver(X, Y, U, V, color='gray', alpha=0.6)

# Plot the parcel trajectory
plt.plot(parcel_trajectory[:, 0], parcel_trajectory[:, 1], label='Parcel Trajectory', marker='None')
plt.scatter([0.6], [0.4], color='blue', s=50, label='Start Point')

plt.xlabel('x')
plt.ylabel('y')
plt.title('Parcel Trajectory on Velocity Field')
plt.legend()
plt.show()

# Many Parcel Simulations - Ocean Parcels library
Now that we have figured out how to interpret fluid advection equations and how to numerically integrate them to get the trajectory of a single parcel in a fluid of plastic particles, it's time to try to numerically integrate the motion of a group of parcels to discover the motion of the fluid of plastic particles as a whole.

As mentioned earlier, Ocean Parcels is the library that this tutorial will walk you through using to achieve this goal. It uses fourth-order Runge-Kutta to numerically integrate the advection equation of parcels, as we just did ourselves and has extra structure so that we can easily apply the same process to large groups of parcels.

The three central objects in Ocean Parcels are:

1) The *FieldSet* - this object stores the information on the velocity field of ocean currents (although the velocity field didn't change with respect to time in our simple example above, it usually does). Other fields containing information about the ocean such as temperature can also be stored here.

2) The *ParticleSet* - this object stores information about the set of particles we are using.

3) The *Kernels* - these objects define the operations carried out at each step of the parcels' numerical integration. In our example above, this would only include the Runge-Kutta step for advection, but it could also include other operations such as random smaller steps to emulate diffusion.

To get an idea of how these objects interact and are used to get our desired results, we'll walk through an example that is already done for you - this example is given on the Ocean Parcels documentation.

To start with, run the following cell to install Ocean Parcels and import all of the neccessary packages:

In [ ]:
%%capture
!pip install parcels
!pip install trajan
!pip install xarray

import math
from datetime import timedelta
from operator import attrgetter

import matplotlib.pyplot as plt
import numpy as np
import trajan as ta
import xarray as xr
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
from matplotlib import rc

import parcels

Now, run the following cell to retrieve a folder of example data from Ocean Parcels. We will use a velocity field from this example data.

In [ ]:
example_dataset_folder = parcels.download_example_dataset("MovingEddies_data")

Now, we can load that data into a *Fieldset* object, using
```parcels.FieldSet.from_parcels()```.
Run the follwing cell to do so:


In [ ]:
fieldset = parcels.FieldSet.from_parcels(f"{example_dataset_folder}/moving_eddies")

It would be nice to view the velocity field - to do this we first have to load a timeframe. To load the first timeframe, we use
```fieldset.computeTimeChunk()```, then we can access the zonal and meridonal velocities (i.e. in the x and y directions in our reference frame) using ```fieldset.U``` and ```fieldset.V```.

The following code uses these methods of the *FieldSet* object to put together two plots of the velocity field - one at an intial time and one at a much later time. We can see that the eddies have moved, so there is some time-dependence in this velocity field.



In [ ]:
plt.figure(figsize=(10, 6))
for t in [0.0, 600000.0]:
  fieldset.computeTimeChunk(time=t)

  # Get the grid and data
  lon = fieldset.U.grid.lon  # 1D array of longitudes
  lat = fieldset.U.grid.lat  # 1D array of latitudes
  U = fieldset.U.data[0, :, :]  # Zonal component (2D array)
  V = fieldset.V.data[0, :, :]  # Meridional component (2D array)

  # Create a 2D meshgrid
  lon_grid, lat_grid = np.meshgrid(lon, lat)

  # Determine the indices to sample
  x_indices = np.linspace(0, lon_grid.shape[1] - 1, 20, dtype=int)
  y_indices = np.linspace(0, lon_grid.shape[0] - 1, 35, dtype=int)

  # Use these indices to select a subset of the data
  quiver_lon = lon_grid[y_indices[:, None], x_indices]
  quiver_lat = lat_grid[y_indices[:, None], x_indices]
  quiver_U = U[y_indices[:, None], x_indices]
  quiver_V = V[y_indices[:, None], x_indices]

  # Create the quiver plot
  if t == 0.0:
    plt.quiver(quiver_lon, quiver_lat, quiver_U, quiver_V, scale=30, color = 'blue', label = 'Initial Time')
  else:
    plt.quiver(quiver_lon, quiver_lat, quiver_U, quiver_V, scale=30, color = 'red', label = 'Later Time')

  # Add labels and title
  plt.xlabel("Zonal distance [m]")
  plt.ylabel("Meridional distance [m]")
  plt.title("Quiver Plot of the Vector Field")
  plt.legend()

  # Optionally set aspect ratio
  plt.gca().set_aspect('equal')

plt.show()


Now, let's put some parcels in this velocity field and track their trajectories.

To set up our *ParticleSet*, we use ```parcels.ParticleSet.from_list()``` which requires four arguments:

1) *fieldset*: the *FieldSet* object that is of relevance.

2) *pclass*: specification of whether to use 'Just In Time' (JIT) or SciPy particles. We'll use JIT particles since just in time computation is a compuationally economic technique and SciPy particles in Ocean Parcels are only neccessary for some more advanced use cases of the library.

3): *lon*: a list of the longitudinal coordinates at which parcels should be released.

4): *lat*: a list of the latitudinal coordinates at which parcels should be released.

(Note that we could also have used ```parcels.ParticleSet.from_line()``` to release the parcels along a line in the 2-d space. In that case, the list of latitudinal and longitudinal start positions would be replaced by *size*, specifying the number of particles, *start*, a tuple *(start_lon,start_lat)* defining the start point of the release line, and *finish*, a tuple *(end_lon,end_lat)* defining the end point of the release line.)

Run the following cell to set this up with two particles:

In [ ]:
pset = parcels.ParticleSet.from_list(
    fieldset=fieldset,  # the fields on which the particles are advected
    pclass=parcels.JITParticle,  # the type of particles (JITParticle or ScipyParticle)
    lon=[3.3e5, 3.3e5],  # a vector of release longitudes
    lat=[1e5, 2.8e5],  # a vector of release latitudes
)

Now, to obtain trajectories run the following cell.

It sets up an output file using ```pset.ParticleFile()```, with one argument, *name*, defining the output file's name and the other, *outputdt*, specifying how often a point on the parcels' trajectories should be saved to the file for later reconstruction.

Then, the numerical integration is executed using ```pset.execute()```, with the first argument defining the *Kernel* (procedure to follow at each timestep), the second argument, *runtime*, defining how long we want to follow the particle's trajectory for, the third argument, *dt*, defining the size of the timestep involved in the numerical integration, and the fourth argument, *output_file*, defining the output file to store the trajectories in.

In [ ]:
output_file = pset.ParticleFile(
    name="EddyParticles.zarr",
    outputdt=timedelta(hours=1),
)
pset.execute(
    parcels.AdvectionRK4,  # this is the 4th order Runge-Kutta Kernel
    runtime=timedelta(days=6),
    dt=timedelta(minutes=5),
    output_file=output_file,
)

We can now plot the trajectories by opening the output using the *xarr* library:

In [ ]:
ds = xr.open_zarr("EddyParticles.zarr")

plt.plot(ds.lon.T, ds.lat.T, ".-")
plt.xlabel("Zonal distance [m]")
plt.ylabel("Meridional distance [m]")
plt.show()

The following cell outputs an animation of the parcels' movement.

In [ ]:
fig = plt.figure(figsize=(3.5, 2.5), constrained_layout=True)
ax = fig.add_subplot()

ax.set_ylabel("Meridional distance [m]")
ax.set_xlabel("Zonal distance [m]")
ax.set_xlim(0, 4e5)
ax.set_ylim(0, 7e5)

# show only every fifth output (for speed in creating the animation)
timerange = np.unique(ds["time"].values)[::5]

# Indices of the data where time = 0
time_id = np.where(ds["time"] == timerange[0])

sc = ax.scatter(ds["lon"].values[time_id], ds["lat"].values[time_id])

t = str(timerange[0].astype("timedelta64[h]"))
title = ax.set_title(f"Particles at t = {t}")

plt.close(fig)

def animate(i):
    t = str(timerange[i].astype("timedelta64[h]"))
    title.set_text(f"Particles at t = {t}")

    time_id = np.where(ds["time"] == timerange[i])
    sc.set_offsets(np.c_[ds["lon"].values[time_id], ds["lat"].values[time_id]])


anim = FuncAnimation(fig, animate, frames=len(timerange), interval=100)
HTML(anim.to_jshtml())

# Using Ocean Parcels - Your Turn

Now that we have seen how to use Ocean Parcels to obtain the trajectories of groups of parcels, let's apply it using real world ocean current data.

The following cell creates a *FieldSet* which stores data about currents in the region around South Africa from [GlobCurrents](https://www.ifremer.fr/fr):

In [ ]:
example_dataset_folder = parcels.download_example_dataset("GlobCurrent_example_data")

filenames = {
    "U": f"{example_dataset_folder}/20*.nc",
    "V": f"{example_dataset_folder}/20*.nc",
}

variables = {
    "U": "eastward_eulerian_current_velocity",
    "V": "northward_eulerian_current_velocity",
}

dimensions = {"lat": "lat", "lon": "lon", "time": "time"}

fieldset = parcels.FieldSet.from_netcdf(filenames, variables, dimensions)

Now, run the following cell to set up the continuous feedback system for this part of the tutorial:

In [ ]:
# @title
import ast
import inspect
from IPython.core.magic import register_cell_magic
from IPython.display import display, HTML

# Define the cell magic function for the particle set
@register_cell_magic
def check_all_3(line, cell):
  # Execute the student's code
    exec(cell, globals())

    correct_pset = parcels.ParticleSet.from_line(
    fieldset=fieldset,
    pclass=parcels.JITParticle,
    size=20,  # releasing 20 particles
    start=(28, -33),  # releasing on a line: the start longitude and latitude
    finish=(30, -33),  # releasing on a line: the end longitude and latitude
    )
    if 'pset_new' in globals():
          if pset_new.fieldset==correct_pset.fieldset:
              print("✅ pset_new's fieldset is correct.")
          else:
              print("❌ pset_new's fieldset should be defined from the one set up in the previous cell.")
          if type(pset_new.particledata.pclass)==type(correct_pset.particledata.pclass):
              print("✅ pset_new's pclass is correct.")
          else:
              print("❌ pset_new's pclass should be JIT.")
          if pset_new.size==correct_pset.size:
              print("✅ pset_new's number of parcels is correct.")
          else:
              print("❌ pset_new's size should be 20.")
          if np.all(pset_new.particledata.lat==correct_pset.particledata.lat):
              print("✅ pset_new's parcel latitudes are correct.")
          elif np.all(pset_new.particledata.lat==correct_pset.particledata.lon):
              print("❌ pset_new's parcel latitudes are incorrect - you may have mixed them up with the desired longitudes.")
          else:
              print("❌ pset_new's parcel latitudes are incorrect.")
          if np.all(pset_new.particledata.lon==correct_pset.particledata.lon):
              print("✅ pset_new's parcel longitudes are correct.")
          elif np.all(pset_new.particledata.lat==correct_pset.particledata.lon):
              print("❌ pset_new's parcel longitudes are incorrect - you may have mixed them up with the desired longitudes.")
          else:
              print("❌ pset_new's parcel longitudes are incorrect.")
    else:
          print("❌ pset_new is not defined.")

# Define the cell magic function for the output file
@register_cell_magic
def check_all_4(line, cell):
    # Execute the student's code
    exec(cell, globals())

    if 'output_file_new' in globals():
          if output_file_new.fname=='output.zarr':
              print("✅ filename is correct.")
          else:
              print("❌ filename is incorrect.")
              print(output_file.fname)
          if output_file_new.outputdt==21600.0:
              print("✅ output time step is correct.")
          else:
              print("❌ output time step is incorrect.")
              print(output_file_new.outputdt)
    else:
          print("❌ output_file is not defined.")

# Define the expected arguments
expected_args = {'runtime': "Call(func=Name(id='timedelta', ctx=Load()), args=[], keywords=[keyword(arg='days', value=Constant(value=10))])", 'dt': "Call(func=Name(id='timedelta', ctx=Load()), args=[], keywords=[keyword(arg='minutes', value=Constant(value=5))])", 'output_file': "Name(id='output_file_new', ctx=Load())"}

def check_arguments(call_node):
    """
    Check if the function call node has the correct arguments.
    """
    args = {kw.arg: ast.dump(kw.value) for kw in call_node.keywords}

    for key, expected_value in expected_args.items():
        if args.get(key) != expected_value:
            return False
    return True

@register_cell_magic
def check_all_5(line, cell):
    # Execute the student's code
    exec(cell, globals())

    # Parse the code cell content
    tree = ast.parse(cell)
    function_called_correctly = False

    # Walk through the AST to find function calls
    for node in ast.walk(tree):
        if isinstance(node, ast.Call):
            if isinstance(node.func, ast.Attribute) and node.func.attr == 'execute':
                if check_arguments(node):
                    function_called_correctly = True
                else:
                    function_called_correctly = False
                break

    # Provide feedback
    if function_called_correctly:
        feedback = "✅ The `pset.execute` function was called with the correct arguments!"
    else:
        feedback = "❌ The `pset.execute` function call does not have the correct arguments. Please check."

    display(HTML(f"<div style='font-size: 14px; color: {'green' if '✅' in feedback else 'red'};'>{feedback}</div>"))

Now, define *pset_new*, a particle set which is defined using ```parcels.ParticleSet.from_line()``` to release twenty particles in a line of constant latitude at -33 and going from 28 to 30 in longitude:

In [ ]:
%%check_all_3 # Leave this here - it will give continuous feedback on your code
#----------------------------------------------------------------------------
#Start coding here:


Now, define *output_file* using ```pset.ParticleFile()```, with the file named 'output.zarr' and the time step between saved positions being 6 hours.

In [ ]:
%%check_all_4 # Leave this here - it will give continuous feedback on your code
#----------------------------------------------------------------------------
#Start coding here:


Finally, execute the numerical integration of particle trajectories for 10 days using the fourth-order Runge-Kutta Kernel with time steps of 5 minutes and saving the trajectories to the output file that you just set up.

In [ ]:
%%check_all_5 # Leave this here - it will give continuous feedback on your code
#----------------------------------------------------------------------------
#Start coding here:


Now we can view the trajectories of the parcels using the following cell:

In [ ]:
ds = xr.open_zarr("output.zarr")
ds.traj.plot(margin=2)
plt.show()

We have now simulated the motion of ocean plastics! The next step to more accurately simulate the trajectories of plastic particles would be to include eddy diffusitivty - the movement of the particle due to currents on scales smaller than the cell size of the Eulerian grid. Including this in the simulation isn't difficult, we would just randomly displace the particle by a small amount at the end of each small step. Beyond this, the inclusion of wind effects and three dimensional models where the depth of particles is also taken into account would add to the precision of the predictions that can be made with the simulations.

If you have enjoyed working through this tutorial, I would recommend having a look at [this article on The Ocean Cleanup's website](https://theoceancleanup.com/updates/forecasting-ocean-plastic-around-the-globe-a-deep-dive-into-modeling-the-garbage-patches/) on the topic and the [OceanParcels documentation](https://docs.oceanparcels.org/en/latest/documentation/index.html) to learn more advanced capabilities of the library - their documentation is a great resource!